```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.
```

# **INTRODUCTIONS**

* * *

The source code is separated into multiple sections:

1. **[Tables](#tables)**
3. **[Stored Procedures](#stored-procedures)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb",
        "src/statics/populations/*",
        "src/statics/calendar.ipynb",
        "src/statics/sewers/*"
    ]
}
```

# **TABLES**

---


### STAGING

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 2) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[RIVM_SEWER_MEASUREMENTS]'))
CREATE TABLE [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[VERSION] [VARCHAR](100) NULL,
	[DATE_MEASUREMENT] [VARCHAR](100) NULL,
	[RWZI_AWZI_CODE] [VARCHAR](100) NULL,
	[RWZI_AWZI_NAME] [VARCHAR](100) NULL,
	[RNA_PER_ML] [VARCHAR](100) NULL,
	[RNA_FLOW_PER_100000] [VARCHAR](100) NULL,
	[DATE_OF_REPORT] [VARCHAR](100) NULL,
	[DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE()
);
GO

### INTERMEDIATES

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[RIVM_SEWER_MEASUREMENTS]'))
CREATE TABLE [VWSINTER].[RIVM_SEWER_MEASUREMENTS] (
    [ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
    [DATE_MEASUREMENT] [DATETIME],
    [VERSION] [VARCHAR](100) NULL,
    [RWZI_AWZI_CODE] [BIGINT],
    [RWZI_AWZI_NAME] [VARCHAR](100) NULL,
    [RNA_PER_ML] [BIGINT],
    [RNA_FLOW_PER_100000] [BIGINT],
    [DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE()
);
GO

### DESTINATIONS

> NATIONAL

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_MEASUREMENTS]'))
CREATE TABLE [VWSDEST].[SEWER_MEASUREMENTS] (
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
	[WEEK_UNIX] BIGINT,
	[AVERAGE_RNA_FLOW_PER_100000] DECIMAL(16,2) NULL,
	[NUMBER_OF_MEASUREMENTS] INT NULL,
	[NUMBER_OF_LOCATIONS] INT NULL,
	[TOTAL_LOCATIONS] INT NULL,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_SEWER_MEASUREMENTS] 
    ON [VWSDEST].[SEWER_MEASUREMENTS]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_SEWER_MEASUREMENTS] 
    ON [VWSDEST].[SEWER_MEASUREMENTS] (
        [DATE_LAST_INSERTED] ASC
    );
GO

> REGIONAL

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_MEASUREMENTS_PER_REGION]'))
CREATE TABLE [VWSDEST].[SEWER_MEASUREMENTS_PER_REGION] (
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
	[WEEK_UNIX] BIGINT,
	[VRCODE] VARCHAR(100),
	[AVERAGE_RNA_FLOW_PER_100000] DECIMAL(16,2) NULL,
	[NUMBER_OF_MEASUREMENTS] INT NULL,
	[NUMBER_OF_LOCATIONS] INT NULL,
	[TOTAL_LOCATIONS] INT NULL,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_SEWER_MEASUREMENTS_PER_REGION] 
    ON [VWSDEST].[SEWER_MEASUREMENTS_PER_REGION]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_SEWER_MEASUREMENTS_PER_REGION] 
    ON [VWSDEST].[SEWER_MEASUREMENTS_PER_REGION] (
        [DATE_LAST_INSERTED] ASC
    );
GO

> MUNICIPALITY

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_MEASUREMENTS_PER_MUNICIPALITY]'))
CREATE TABLE [VWSDEST].[SEWER_MEASUREMENTS_PER_MUNICIPALITY] (
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
	[WEEK_UNIX] BIGINT,
	[GMCODE] VARCHAR(100),
	[AVERAGE_RNA_FLOW_PER_100000] DECIMAL(16,2) NULL,
	[NUMBER_OF_MEASUREMENTS] INT NULL,
	[NUMBER_OF_LOCATIONS] INT NULL,
	[TOTAL_LOCATIONS] INT NULL,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_SEWER_MEASUREMENTS_PER_MUNICIPALITY] 
    ON [VWSDEST].[SEWER_MEASUREMENTS_PER_MUNICIPALITY]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_SEWER_MEASUREMENTS_PER_MUNICIPALITY] 
    ON [VWSDEST].[SEWER_MEASUREMENTS_PER_MUNICIPALITY] (
        [DATE_LAST_INSERTED] ASC
    );
GO

> SEWER BASE

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_BASE]'))
CREATE TABLE [VWSDEST].[SEWER_BASE] (
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
	[WEEK] INT NULL,
    [WEEK_UNIX] BIGINT NULL,
    [DATE_MEASUREMENT] DATETIME NULL,
    [RWZI_AWZI_CODE] INT NULL,
    [RWZI_AWZI_NAME] VARCHAR(100) NULL,
    [GM_CODE] VARCHAR(100) NULL,
    [RNA_PER_ML] BIGINT,
    [RNA_FLOW_PER_100000] BIGINT NULL,
    [COUNT_DAILY_MEASUREMENT] INT NULL,
    [DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_SEWER_BASE] 
    ON [VWSDEST].[SEWER_BASE]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_SEWER_BASE] 
    ON [VWSDEST].[SEWER_BASE] (
        [DATE_LAST_INSERTED] ASC
    );
GO

> SEWER PROLONGATED

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_PROLONGATED]'))
CREATE TABLE [VWSDEST].[SEWER_PROLONGATED](
  [ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
  [DATE_MEASUREMENT] [DATETIME] NULL,               
  [REGIO_CODE] [VARCHAR](10) NULL,          
  [WEEK] [INT] NULL,          
  [WEEK_UNIX] [BIGINT] NULL,      
  [AVERAGE_RNA_FLOW_PER_100000] [DECIMAL](16,3) NULL,          
  [REGION_COVERED_INHABITANTS] [INT] NULL,                                                              
  [DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_DM_RC_SEWER_PROLONGATED] 
  ON [VWSDEST].[SEWER_PROLONGATED]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_DM_RC_SEWER_PROLONGATED] 
  ON [VWSDEST].[SEWER_PROLONGATED] (
    [DATE_LAST_INSERTED],
    [DATE_MEASUREMENT],
    [REGIO_CODE]
  );
GO

> SEWER PER RWZI

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI]'))
CREATE TABLE [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI] (
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
	[DATE_MEASUREMENT] DATETIME,
	[DATE_MEASUREMENT_UNIX] BIGINT NULL,
	[WEEK] INT NULL,
	[RWZI_AWZI_CODE] VARCHAR(100) NULL,
	[RWZI_AWZI_NAME] VARCHAR(100) NULL,
	[RNA_FLOW_PER_100000] DECIMAL(16,2) NULL,
	[AVERAGE_RNA_FLOW_PER_100000] DECIMAL(16,2) NULL,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_RWC_SEWER_MEASUREMENTS_PER_RWZI] 
	ON [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RWC_SEWER_MEASUREMENTS_PER_RWZI] 
	ON [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI] (
		[DATE_LAST_INSERTED],
		[RWZI_AWZI_CODE]
	);
GO

> SEWER PER RWZI MINIMAL

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI_MINIMAL]'))
CREATE TABLE [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI_MINIMAL](
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
	[DATE_UNIX] [bigint] NULL,
	[DATE_START_UNIX] [bigint] NULL,
	[DATE_END_UNIX] [bigint] NULL,
	[WEEK] [int] NULL,
	[RWZI_AWZI_CODE] [varchar](100) NULL,
	[RWZI_AWZI_NAME] [varchar](100) NULL,
	[GMCODE] [varchar](100) NULL,
	[RNA_NORMALIZED] [decimal](16, 2) NULL,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
) ON [PRIMARY]
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_RWC_SEWER_MEASUREMENTS_PER_RWZI_MINIMAL] 
	ON [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI_MINIMAL]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RWC_SEWER_MEASUREMENTS_PER_RWZI_MINIMAL] 
	ON [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI_MINIMAL] (
		[DATE_LAST_INSERTED],
		[RWZI_AWZI_CODE]
	);
GO


> SEWER PER RWZI MUNICIPALITIES & SAFETY REGIONS

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI_GM_VR]'))
CREATE TABLE [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI_GM_VR] (
	[ID] BIGINT PRIMARY KEY IDENTITY(1,1),
        [DATE_MEASUREMENT] DATETIME,	
	[RWZI_AWZI_CODE] VARCHAR(100) NULL,
	[RWZI_AWZI_NAME] VARCHAR(100) NULL,
	[RNA_FLOW_PER_100000] DECIMAL(16,2) NULL,					
        [REGIO_TYPE] VARCHAR(10) NULL,									
        [REGIO_CODE] VARCHAR(10) NULL,										
        [WEEK] INT NULL,														
        [DATE_UNIX] BIGINT,        
        [DATE_START_UNIX] BIGINT,        
        [DATE_END_UNIX] BIGINT,        
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_RT_RC_SEWER_MEASUREMENTS_PER_RWZI_GM_VR]
        ON [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI_GM_VR]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RT_RC_SEWER_MEASUREMENTS_PER_RWZI_GM_VR]
        ON [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI_GM_VR] (
                [DATE_LAST_INSERTED],
                [REGIO_TYPE],
                [REGIO_CODE]
        )                
        INCLUDE (
                [RWZI_AWZI_CODE],
                [RWZI_AWZI_NAME],
                [RNA_FLOW_PER_100000],										
                [WEEK],														
                [DATE_UNIX],        
                [DATE_START_UNIX],        
                [DATE_END_UNIX]             
        );
GO

# **VIEWS**

---

> NATIONAL

In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_SEWER_MEASUREMENTS]
AS
  WITH Sel As (
      SELECT 
          dbo.CONVERT_DATETIME_TO_UNIX([DATE_MEASUREMENT]) AS DATE_UNIX
          , CAST(round([AVERAGE_RNA_FLOW_PER_100000],0) as INT)          AS [AVERAGE]
          ,dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS [DATE_OF_INSERTION_UNIX]
          ,Regio_Code
          ,date_Last_Inserted
      FROM [VWSDEST].[SEWER_PROLONGATED]
      -- First filter on Wednesdays
      WHERE DatePart(WeekDay, Date_Last_Inserted) IN(3, 4, 6)
  )
  SELECT 
        DATE_UNIX
        ,[AVERAGE]
        ,[DATE_OF_INSERTION_UNIX]
    FROM Sel
    -- Then filter on the newest dataset
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM Sel)
      AND REGIO_CODE = 'NL'
GO

In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_SEWER__AVERAGE] 
AS
  WITH CTE 
  AS (
    SELECT
      [DBO].[CONVERT_DATETIME_TO_UNIX]([DATE_MEASUREMENT]) AS [DATE_UNIX]
      ,[DBO].[CONVERT_DATETIME_TO_UNIX](LAG(DATE_MEASUREMENT,1) OVER (PARTITION BY [REGIO_CODE] ORDER BY [DATE_MEASUREMENT])) AS [OLD_DATE_UNIX]
      ,CAST(ROUND([AVERAGE_RNA_FLOW_PER_100000],0) AS INT) AS [NEW_VALUE]
      ,CAST(ROUND(LAG([AVERAGE_RNA_FLOW_PER_100000],1) OVER (PARTITION BY [REGIO_CODE] ORDER BY [DATE_MEASUREMENT]),0) AS INT) AS [OLD_VALUE]
      ,[DBO].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS [DATE_OF_INSERTION_UNIX]
      ,ROW_NUMBER() OVER (PARTITION BY [REGIO_CODE] ORDER BY [DATE_MEASUREMENT] DESC) AS [RANK_DATE]
    FROM [VWSDEST].[SEWER_PROLONGATED] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[SEWER_PROLONGATED] WITH(NOLOCK))
      AND UPPER([REGIO_CODE]) = 'NL'
  )
  SELECT
    [DATE_UNIX] AS [NEW_DATE_UNIX],
    [OLD_DATE_UNIX],
    [OLD_VALUE],
    ([NEW_VALUE] - [OLD_VALUE]) AS [DIFFERENCE]
  FROM CTE
  WHERE [RANK_DATE] = 1 -- ONLY THE MOST RECENT RECORD

> REGIONAL

In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_SEWER_MEASUREMENTS_PER_REGION] AS

  WITH Sel As (
  SELECT
        REGIO_CODE                                               AS VRCODE
        ,dbo.CONVERT_DATETIME_TO_UNIX([DATE_MEASUREMENT])         AS DATE_UNIX
        ,CAST(ROUND([AVERAGE_RNA_FLOW_PER_100000],0) as INT)      AS [AVERAGE]
        ,dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED)         AS [DATE_OF_INSERTION_UNIX]
        ,Date_Last_inserted
    FROM [VWSDEST].[SEWER_PROLONGATED]
    WHERE DatePart(WeekDay,[DATE_LAST_INSERTED]) IN(3, 4, 6)
      AND REGIO_CODE <> 'NL'

  )
  SELECT VRCODE
        , DATE_UNIX
        , [AVERAGE]
        , [DATE_OF_INSERTION_UNIX]
    FROM Sel
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM Sel)
GO

In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_SEWER__AVERAGE_PER_REGION] 
AS
  WITH CTE 
  AS (
    SELECT
      [DATE_MEASUREMENT]
      ,[REGIO_CODE]                                                                                                             AS [VRCODE]
      ,DBO.CONVERT_DATETIME_TO_UNIX([DATE_MEASUREMENT])                                                                         AS DATE_UNIX
      ,DBO.CONVERT_DATETIME_TO_UNIX(LAG(DATE_MEASUREMENT,1) OVER (PARTITION BY [REGIO_CODE] ORDER BY DATE_MEASUREMENT))         AS OLD_DATE_UNIX
      ,CAST(ROUND([AVERAGE_RNA_FLOW_PER_100000],0)  AS INT)                                                                     AS [NEW_VALUE]
      ,CAST(ROUND(LAG([AVERAGE_RNA_FLOW_PER_100000],1) OVER (PARTITION BY [REGIO_CODE] ORDER BY DATE_MEASUREMENT),0) AS INT)    AS OLD_VALUE
      ,DBO.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED)                                                                         AS [DATE_OF_INSERTION_UNIX]
      ,ROW_NUMBER() OVER (PARTITION BY [REGIO_CODE] ORDER BY DATE_MEASUREMENT DESC)                     AS RANK_DATE
    FROM [VWSDEST].[SEWER_PROLONGATED] WITH(NOLOCK)
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[SEWER_PROLONGATED] WITH(NOLOCK))
      AND UPPER([REGIO_CODE]) <> 'NL'
  )
  SELECT
    DATE_UNIX AS [NEW_DATE_UNIX]
    ,OLD_DATE_UNIX
    ,OLD_VALUE
    ,NEW_VALUE - OLD_VALUE AS [DIFFERENCE]
    ,VRCODE
  FROM CTE
    WHERE RANK_DATE = 1 -- ONLY THE MOST RECENT RECORD
GO

> REGIONAL COLLECTION

In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_SEWER_MEASUREMENTS_REGIONS] 
AS
  SELECT
    [VRCODE],
    [DATE_UNIX],
    [AVERAGE],
    [DATE_OF_INSERTION_UNIX]
  FROM (
      SELECT
        [VRCODE],
        [DATE_UNIX],
        [AVERAGE],
        [DATE_OF_INSERTION_UNIX],
        RANK() OVER (PARTITION BY [VRCODE] ORDER BY [DATE_UNIX] DESC) [RANKING]
      FROM [VWSDEST].[V_SEWER_MEASUREMENTS_PER_REGION]
  ) T1
  WHERE T1.[RANKING] = 1
GO

> MUNICIPALITY

In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_SEWER_MEASUREMENTS_PER_MUNICIPALITY] AS

WITH Sel As (
    SELECT
        dbo.CONVERT_DATETIME_TO_UNIX(dbo.WEEK_START(dbo.CONVERT_UNIX_TO_DATETIME(WEEK_UNIX))) AS DATE_START_UNIX
    ,   dbo.CONVERT_DATETIME_TO_UNIX(dbo.WEEK_END(dbo.CONVERT_UNIX_TO_DATETIME(WEEK_UNIX))) AS DATE_END_UNIX
    ,   CASE WHEN dbo.WEEK_START(dbo.CONVERT_UNIX_TO_DATETIME(WEEK_UNIX)) < dateadd(day, -14, dbo.WEEK_START(DATE_LAST_INSERTED)) THEN 1 ELSE 0 END AS data_is_outdated
    ,   GMCODE
    ,   CAST(ROUND(AVERAGE_RNA_FLOW_PER_100000,0) as INT) AS AVERAGE
    ,   NUMBER_OF_MEASUREMENTS AS TOTAL_NUMBER_OF_SAMPLES
    ,   NUMBER_OF_LOCATIONS AS SAMPLED_INSTALLATION_COUNT
    ,   TOTAL_LOCATIONS AS TOTAL_INSTALLATION_COUNT
    ,   dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
    ,   Date_Last_Inserted
FROM VWSDEST.SEWER_MEASUREMENTS_PER_MUNICIPALITY 
WHERE  DatePart(WeekDay,Date_Last_Inserted) IN(3, 4, 6) -- First filter on only Wednesdays
-- week_unix is on monday. We can only publish the data of that week after tuesday of next week.
AND GETDATE() > dateadd(day, 8,dateadd(S, WEEK_UNIX, '1970-01-01'))
AND dateadd(S, WEEK_UNIX, '1970-01-01') > '2020-01-01'
)
SELECT Date_Start_Unix
     , Date_End_Unix
     , data_is_outdated
     , GMCode
     , AVERAGE 
     , TOTAL_NUMBER_OF_SAMPLES 
     , SAMPLED_INSTALLATION_COUNT 
     , TOTAL_INSTALLATION_COUNT 
     , DATE_OF_INSERTION_UNIX 
FROM Sel
-- Then filter on newest dataset
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM Sel)

GO

> MUNICIPALITY COLLECTION

In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_SEWER_MEASUREMENTS_MUNICIPALITIES] 
AS
    SELECT 
        [DATE_START_UNIX],
        [DATE_END_UNIX],
        [GMCODE],
        [AVERAGE],
        [TOTAL_INSTALLATION_COUNT],
        [DATA_IS_OUTDATED],
        [DATE_OF_INSERTION_UNIX]
    FROM (
    SELECT
        [DATE_START_UNIX],
        [DATE_END_UNIX],
        [GMCODE],
        [AVERAGE],
        [TOTAL_INSTALLATION_COUNT],
        [DATA_IS_OUTDATED],
        [DATE_OF_INSERTION_UNIX],
        RANK() OVER (PARTITION BY [GMCODE] ORDER BY [DATE_START_UNIX] DESC) [RANKING]
    FROM [VWSDEST].[V_SEWER_MEASUREMENTS_PER_MUNICIPALITY]
    ) T1
    WHERE T1.[RANKING] = 1 AND [GMCODE] IS NOT NULL
GO

> DIFFERENCES PER MUNCIPALITY

In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_SEWER__AVERAGE_PER_MUNICIPALITY] 
AS
	WITH PREPARATION AS (
		SELECT
			[WEEK_UNIX],
			DBO.CONVERT_DATETIME_TO_UNIX(DBO.WEEK_START(DBO.CONVERT_UNIX_TO_DATETIME(WEEK_UNIX))) AS WEEK_START_UNIX,
			DBO.CONVERT_DATETIME_TO_UNIX(DBO.WEEK_END(DBO.CONVERT_UNIX_TO_DATETIME(WEEK_UNIX))) AS WEEK_END_UNIX,
			GMCODE,
			CAST(ROUND(AVERAGE_RNA_FLOW_PER_100000,0) AS INT) AS AVERAGE,
			DBO.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
		FROM VWSDEST.SEWER_MEASUREMENTS_PER_MUNICIPALITY
		WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.SEWER_MEASUREMENTS_PER_MUNICIPALITY)
		-- WEEK_UNIX IS ON MONDAY. WE CAN ONLY PUBLISH THE DATA OF THAT WEEK AFTER TUESDAY OF NEXT WEEK.
		AND GETDATE() > DATEADD(DAY, 8,DATEADD(S, WEEK_UNIX, '1970-01-01'))
			AND dateadd(S, WEEK_UNIX, '1970-01-01') > '2020-01-01'

	),

LAST_DATE_OF_REPORT AS (
	SELECT
		GMCODE,
		MAX(WEEK_UNIX) AS [LAST_DATE_OF_REPORT]
	FROM PREPARATION
	GROUP BY GMCODE
),


BASE_CTE AS (
SELECT
    WEEK_UNIX,
	GMCODE,
	LAG([WEEK_UNIX], 1, NULL) OVER (PARTITION BY [GMCODE] ORDER BY [WEEK_UNIX]) AS [OLD_DATE_OF_REPORT_UNIX],
	LAG([AVERAGE], 1, NULL) OVER (PARTITION BY [GMCODE] ORDER BY [WEEK_UNIX]) AS [OLD_VALUE],
    [AVERAGE] -
		LAG([AVERAGE], 1, NULL) OVER (PARTITION BY [GMCODE] ORDER BY [WEEK_UNIX]) AS [DIFFERENCE]
--  DBO.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM PREPARATION

)

SELECT WEEK_UNIX AS [NEW_DATE_UNIX]
	  ,T1.GMCODE
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,CASE WHEN OLD_VALUE IS NULL THEN 0 ELSE OLD_VALUE END AS OLD_VALUE
	  ,CASE WHEN [DIFFERENCE] IS NULL THEN 0 ELSE [DIFFERENCE] END AS [DIFFERENCE]
FROM BASE_CTE T1
LEFT JOIN LAST_DATE_OF_REPORT T2
	ON T1.[GMCODE] = T2.[GMCODE]
WHERE WEEK_UNIX = LAST_DATE_OF_REPORT
;
GO




> SEWER PER RWZI MUNICPALITY

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

CREATE OR ALTER VIEW [VWSDEST].[V_SEWER_MEASUREMENTS_PER_RWZI_MINIMAL] As

WITH Sel As (
SELECT
            [DATE_UNIX]
        ,   DATE_START_UNIX
        ,   DATE_END_UNIX
        ,   WEEK
        ,   [RWZI_AWZI_CODE]
        ,   UPPER([RWZI_AWZI_NAME]) AS [RWZI_AWZI_NAME]
        ,   REGIO_CODE          AS [GMCODE]
        ,   CAST(ROUND(RNA_FLOW_PER_100000,0) AS INT) AS RNA_NORMALIZED
        ,   dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
        ,   Date_Last_Inserted
FROM VWSDEST.SEWER_MEASUREMENTS_PER_RWZI_GM_VR WITH(NOLOCK)
    WHERE DatePart(WeekDay,DATE_LAST_INSERTED) IN(3, 4, 6)
        AND REGIO_TYPE = 'GM'
)
SELECT
            [DATE_UNIX]
        ,   DATE_START_UNIX
        ,   DATE_END_UNIX
        ,   WEEK
        ,   [RWZI_AWZI_CODE]
        ,   [RWZI_AWZI_NAME]
        ,   [GMCODE]
        ,   RNA_NORMALIZED
        ,   DATE_OF_INSERTION_UNIX
FROM Sel
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM Sel)
        



> SEWER PER RWZI REGIONAL

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

-- View for VR
CREATE OR ALTER  VIEW [VWSDEST].[V_SEWER_MEASUREMENTS_PER_RWZI] AS

WITH Sel As (
  SELECT
            [DATE_UNIX]
        ,   DATE_START_UNIX
        ,   DATE_END_UNIX
        ,   WEEK
        ,   [RWZI_AWZI_CODE]
        ,   UPPER([RWZI_AWZI_NAME]) AS [RWZI_AWZI_NAME]
        ,   REGIO_CODE          AS[VRCODE]
        ,   CAST(ROUND(RNA_FLOW_PER_100000,0) as INT) AS RNA_NORMALIZED
        ,   dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
        ,   Date_Last_Inserted
FROM VWSDEST.SEWER_MEASUREMENTS_PER_RWZI_GM_VR WITH(NOLOCK)
    WHERE DatePart(WeekDay,DATE_LAST_INSERTED) IN(3, 4, 6)
        AND REGIO_TYPE = 'VR'
)
  SELECT
            [DATE_UNIX]
        ,   DATE_START_UNIX
        ,   DATE_END_UNIX
        ,   WEEK
        ,   [RWZI_AWZI_CODE]
        ,   [RWZI_AWZI_NAME]
        ,   [VRCODE]
        ,   RNA_NORMALIZED
        ,   DATE_OF_INSERTION_UNIX
FROM Sel
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM Sel)
GO

# **STORED PROCEDURES**

---


### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- MOVE SEWER MEASUREMENTS DATA FROM STAGING TO INTERMEDIATE TABLE.

-- COMMENTS:
-- 1. EITHER RNA_PER_ML OR RNA_FLOW_PER_100000 IS FILLED. THE OTHER IS EMPTY: RNA_PER_ML SEEMS LEGACY
-- 2. RNA_FLOW_PER_100000 CAN BE EITHER EMPTY OR 0. RECORDS WITH NO VALUES ARE LEFT OUT (RNA_PER_ML WAS MEASURED)

CREATE OR ALTER PROCEDURE [DBO].[SP_RIVM_SEWER_MEASUREMENTS_INTER]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_SEWER_MEASUREMENTS] (
        T1.[DATE_MEASUREMENT],
        T1.[VERSION],
        T1.[RWZI_AWZI_CODE],
        T1.[RWZI_AWZI_NAME],
        T1.[RNA_PER_ML],
        T1.[RNA_FLOW_PER_100000]
    )
    SELECT 
        [DBO].[TRY_CONVERT_TO_DATETIME](T1.[DATE_MEASUREMENT]) AS [DATE_MEASUREMENT],
        T1.[VERSION],
        T1.[RWZI_AWZI_CODE],
        T1.[RWZI_AWZI_NAME],
        CAST((SELECT CONVERT(FLOAT, ISNULL(NULLIF(T1.[RNA_PER_ML], ''), '0'))) AS BIGINT) AS [RNA_PER_ML],
        CONVERT(BIGINT, CONVERT(FLOAT, REPLACE(T1.[RNA_FLOW_PER_100000],',','.'))) AS [RNA_FLOW_PER_100000]
    FROM 
        [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS] T1
    LEFT JOIN [VWSSTATIC].[INACTIVE_SEWER_INSTALLATION_MAPPING] T2
        ON CAST(T1.[RWZI_AWZI_CODE] AS INT) = T2.[RWZI_CODE]
    WHERE T1.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS])
        AND T1.[RNA_FLOW_PER_100000] != ''
        AND ISNULL(T2.[ACTIVE], 1) != 0
END;
GO

### INTERMEDIATE &rarr; DESTINATION MAPPING

> SEWER BASE

In [ ]:
CREATE OR ALTER PROCEDURE [DBO].[SP_SEWER_BASE]
AS
BEGIN
    INSERT INTO [VWSDEST].[SEWER_BASE] (
        [WEEK],
        [WEEK_UNIX],
        [DATE_MEASUREMENT],
        [RWZI_AWZI_CODE],
        [RWZI_AWZI_NAME],
        [RNA_PER_ML],
        [RNA_FLOW_PER_100000],
        [COUNT_DAILY_MEASUREMENT]
    )
    SELECT
        DATEPART(ISO_WEEK, T1.[DATE_MEASUREMENT]) AS [WEEK],
        [DBO].[CONVERT_DATETIME_TO_UNIX](DATEADD(week, DATEDIFF(week, 0, T1.[DATE_MEASUREMENT] - 1), 0)) AS [WEEK_UNIX],
        T1.[DATE_MEASUREMENT],
        T1.[RWZI_AWZI_CODE],
        T1.[RWZI_AWZI_NAME],
        T1.[RNA_PER_ML],
        T1.[RNA_FLOW_PER_100000],
        -- SO FIRST OCCURENCE OF A DATE_MEASUREMENT, RWZI_AWZI_CODE GETS A 1, SECOND 2.
        -- WE DO THIS BECAUSE WE FILTER ON THIS LATER
        ROW_NUMBER() OVER (PARTITION BY [DATE_MEASUREMENT], [RWZI_AWZI_CODE] ORDER BY [DATE_MEASUREMENT], [RWZI_AWZI_CODE]) AS [COUNT_DAILY_MEASUREMENT]
    FROM [VWSINTER].[RIVM_SEWER_MEASUREMENTS] AS T1
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSINTER].[RIVM_SEWER_MEASUREMENTS] WITH(NOLOCK))
        AND [DATE_MEASUREMENT] >= CAST('2020-09-07 00:00:00.000' AS DATETIME)
        AND TRIM(UPPER([RWZI_AWZI_NAME])) != 'SCHIPHOL' 
END
GO

> SEWER PER RWZI

In [ ]:
-- MOVE SEWER MEASUREMENTS DATA PER SEWER TREATMENT PLANT FROM INTERMEDIATE TO PRODUCTION TABLE.
CREATE OR ALTER PROCEDURE [DBO].[SP_SEWER_MEASUREMENTS_PER_RWZI]
AS
BEGIN
    -- ONLY TAKE ROWS INTO ACCOUNT WHICH ARE REPRESENTATIVE MEASUREMENTS AND REMOVE THE ROWS FROM PLANT 'SCHIPHOL'.
    WITH BASE_CTE AS (
        SELECT
            DATEPART(wk, [DATE_MEASUREMENT]) AS [WEEK]
            ,[DBO].[CONVERT_DATETIME_TO_UNIX]([DATE_MEASUREMENT]) AS [DATE_MEASUREMENT_UNIX]
            ,[DATE_MEASUREMENT]
            ,CAST([RNA_FLOW_PER_100000] / 1.0E+11 AS DECIMAL(16,2)) AS [RNA_FLOW_PER_100000]
            ,[RWZI_AWZI_CODE]
        FROM [VWSINTER].[RIVM_SEWER_MEASUREMENTS]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_SEWER_MEASUREMENTS])
        AND TRIM(UPPER([RWZI_AWZI_NAME])) != 'SCHIPHOL'
    ),
    RECENT_RWZI_AWZI AS (
        SELECT
            [RWZI_AWZI_CODE],
            [RWZI_AWZI_NAME]
        FROM [VWSSTATIC].[RWZI_AWZI]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RWZI_AWZI] )
    )
    INSERT INTO [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI] (   
        [DATE_MEASUREMENT]
        ,[DATE_MEASUREMENT_UNIX]
        ,[WEEK]
        ,[RWZI_AWZI_CODE]
        ,[RWZI_AWZI_NAME]
        ,[RNA_FLOW_PER_100000]
        )
    SELECT
        [DATE_MEASUREMENT]
        ,[DATE_MEASUREMENT_UNIX]
        ,[WEEK]
        ,T1.[RWZI_AWZI_CODE]
        ,[RWZI_AWZI_NAME]
        ,[RNA_FLOW_PER_100000]
        FROM BASE_CTE T1
            LEFT JOIN RECENT_RWZI_AWZI T2 ON T1.[RWZI_AWZI_CODE]= T2.[RWZI_AWZI_CODE]
END;
GO

### DESTINATION &rarr; DESTINATION MAPPING

> NATIONAL

In [ ]:
-- CREATE VIEW(S).....
CREATE OR ALTER PROCEDURE [dbo].[SP_SEWER_NL]
AS
BEGIN
    WITH BASE_CTE AS (
        SELECT WEEK_GEM_PER_RWZI.*
        ,CASE
            WHEN WEEK_GEM_PER_RWZI.RWZI_AWZI_CODE = 9041 AND WEEK_UNIX < 1601856000 THEN 41974 -- Value for Zaltbommel before closing Aalst
            WHEN WEEK_GEM_PER_RWZI.RWZI_AWZI_CODE = 9025 AND WEEK_UNIX < 1607299200 THEN 56015 -- Value for Tiel before closing Lienden
            ELSE INHABITANTS
        END AS INHABITANTS
        ,CASE
            WHEN WEEK_GEM_PER_RWZI.RWZI_AWZI_CODE = 9041 AND WEEK_UNIX < 1601856000
                THEN (AVG_RNA_FLOW_PER_100000_PER_RWZI / 100000 ) * 41974 -- Value for Zaltbommel before closing Aalst
            WHEN WEEK_GEM_PER_RWZI.RWZI_AWZI_CODE = 9025 AND WEEK_UNIX < 1607299200
                THEN (AVG_RNA_FLOW_PER_100000_PER_RWZI / 100000 ) * 56015 -- Value for Tiel before closing Lienden
            ELSE (AVG_RNA_FLOW_PER_100000_PER_RWZI / 100000 ) * INHABITANTS
        END AS AVG_RNA_FLOW_PER_RWZI
        FROM (
            SELECT
            SB.RWZI_AWZI_CODE,
            WEEK_UNIX,
            AVG(RNA_FLOW_PER_100000) / 1.0E+11 as AVG_RNA_FLOW_PER_100000_PER_RWZI
            , count(*) AS NUMBER_OF_MEASUREMENTS
            FROM VWSDEST.SEWER_BASE SB
            where DATE_LAST_INSERTED = (select max(DATE_LAST_INSERTED) from VWSDEST.SEWER_BASE)
            -- keep only the first measurement (some duplicate measurements per day per RWZI are present)
            and COUNT_DAILY_MEASUREMENT=1
            -- Take the average of the measurements for 1 RWZI for 1 week
            group by RWZI_AWZI_CODE, WEEK_UNIX
        ) WEEK_GEM_PER_RWZI
        LEFT JOIN (
            SELECT DISTINCT CODE_RWZI, INHABITANTS
            FROM [VWSSTATIC].[RWZI_INHIBITANTS_2021]
            WHERE DATE_LAST_INSERTED = (select max(DATE_LAST_INSERTED) from [VWSSTATIC].[RWZI_INHIBITANTS_2021])
            -- Add 2 removed RWZI with 2020 data
            UNION SELECT 9029, 10495    -- Aalst
            UNION SELECT 9043, 6728     -- Lienden
        ) AS V
            ON WEEK_GEM_PER_RWZI.RWZI_AWZI_CODE = V.CODE_RWZI
    )
    INSERT INTO VWSDEST.SEWER_MEASUREMENTS(
            WEEK_UNIX
        ,   AVERAGE_RNA_FLOW_PER_100000
        ,   NUMBER_OF_MEASUREMENTS
        ,   NUMBER_OF_LOCATIONS
        ,   TOTAL_LOCATIONS
    )
    SELECT
        WEEK_UNIX
    ,   ROUND((SUM(AVG_RNA_FLOW_PER_RWZI) / SUM(INHABITANTS)) * 100000 , 2) as AVERAGE_RNA_FLOW_PER_100000
    ,   SUM(NUMBER_OF_MEASUREMENTS) as NUMBER_OF_MEASUREMENTS
    ,   count(*) AS NUMBER_OF_LOCATIONS
    ,   CASE
            WHEN WEEK_UNIX < 1601856000 THEN 317 -- Vanaf week van 5-10-2020 is RWZI Aalst (code 9029) opgeheven
            WHEN WEEK_UNIX >= 1607299200 THEN 315 -- Vanaf week van 7-12-2020 is RWZI Lienden (code 9043) opgeheven
            ELSE 316
        END AS TOTAL_NUMBER_OF_LOCATIONS
    FROM BASE_CTE group by WEEK_UNIX
END
GO

> REGIONAL

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER PROCEDURE [dbo].[SP_SEWER_VR]
AS
BEGIN
-- Calculate percentages and inhabitants for each RWZI for each VR
WITH Percentages AS (
        SELECT CODE_RWZI
        ,GEBIEDSCODE
        ,INHABITANTS
        ,Percentage / 100 AS PERCENTAGE
        FROM [VWSSTATIC].[RWZI_INHIBITANTS_2021]
        WHERE GEBIEDSCODE LIKE 'VR%'
        AND DATE_LAST_INSERTED = (select max(DATE_LAST_INSERTED) from [VWSSTATIC].[RWZI_INHIBITANTS_2021])
        AND PERCENTAGE != 0
        -- Add exceptions
        -- Add Aalst
        UNION SELECT 9029, 'VR08', 10495, 1.00
        -- Add Lienden
        UNION SELECT 9043, 'VR08', 6728, 1.00
)
-- Calculate average per RWZI per week
,BASE_CTE AS (
    SELECT WEEK_GEM_PER_RWZI.*
    ,GEBIEDSCODE
    ,CASE
        -- Adjust for Zaltbommel
        WHEN RWZI_AWZI_CODE = 9041 AND WEEK_UNIX < 1601856000 THEN (41974 * PERCENTAGE)
        -- Adjust for Tiel
        WHEN RWZI_AWZI_CODE = 9025 AND WEEK_UNIX < 1607299200 THEN (56015 * PERCENTAGE)
        ELSE (INHABITANTS * PERCENTAGE)
    END AS APPLICABLE_INHABITANTS
    ,CASE
        -- Adjust for Zaltbommel
        WHEN RWZI_AWZI_CODE = 9041 AND WEEK_UNIX < 1601856000 THEN ((AVG_RNA_FLOW_PER_100000_PER_RWZI / 100000 ) * (41974 * PERCENTAGE))
        -- Adjust for Tiel
        WHEN RWZI_AWZI_CODE = 9025 AND WEEK_UNIX < 1607299200 THEN ((AVG_RNA_FLOW_PER_100000_PER_RWZI / 100000 ) * (56015 * PERCENTAGE))
        ELSE (AVG_RNA_FLOW_PER_100000_PER_RWZI / 100000 ) * (INHABITANTS * PERCENTAGE)
    END AS AVG_RNA_FLOW_PER_RWZI
    FROM (
        SELECT
        SB.RWZI_AWZI_CODE,
        WEEK_UNIX,
        AVG(RNA_FLOW_PER_100000) / 1.0E+11 as AVG_RNA_FLOW_PER_100000_PER_RWZI
        , count(*) AS NUMBER_OF_MEASUREMENTS
        FROM VWSDEST.SEWER_BASE SB
        where DATE_LAST_INSERTED = (select max(DATE_LAST_INSERTED) from VWSDEST.SEWER_BASE)
        -- keep only the first measurement (some duplicate measurements per day per RWZI are present)
        and COUNT_DAILY_MEASUREMENT=1
        -- Take the average of the measurements for 1 RWZI for 1 week
        group by RWZI_AWZI_CODE, WEEK_UNIX
    ) WEEK_GEM_PER_RWZI
    -- Add percentages and inhabitants for each RWZI for each VR
    LEFT JOIN Percentages
        ON WEEK_GEM_PER_RWZI.RWZI_AWZI_CODE = Percentages.CODE_RWZI
)

INSERT INTO VWSDEST.SEWER_MEASUREMENTS_PER_REGION(
        WEEK_UNIX
    ,   VRCODE
    ,   AVERAGE_RNA_FLOW_PER_100000
    ,   NUMBER_OF_MEASUREMENTS
    ,   NUMBER_OF_LOCATIONS
    ,   TOTAL_LOCATIONS
)

SELECT
    WEEK_UNIX
,   B.GEBIEDSCODE
,   ROUND((SUM(AVG_RNA_FLOW_PER_RWZI) / SUM(APPLICABLE_INHABITANTS)) * 100000 , 2) as AVERAGE_RNA_FLOW_PER_100000
,   SUM(NUMBER_OF_MEASUREMENTS) as NUMBER_OF_MEASUREMENTS
,   count(*) AS NUMBER_OF_LOCATIONS
,   CASE
        WHEN B.GEBIEDSCODE = 'VR08' AND WEEK_UNIX < 1601856000 THEN MAX(NUMBER_OF_RWZI)+2       -- Vanaf week van 5-10-2020 is RWZI Aalst (code 9029) opgeheven
        WHEN B.GEBIEDSCODE = 'VR08' AND WEEK_UNIX >= 1601856000 AND  WEEK_UNIX < 1607299200
            THEN MAX(NUMBER_OF_RWZI)+1                                                          -- Vanaf week van 7-12-2020 is RWZI Lienden (code 9043) opgeheven
        ELSE MAX(NUMBER_OF_RWZI)                                                                -- Number of 2021
     END AS TOTAL_NUMBER_OF_LOCATIONS
FROM BASE_CTE  B
-- Add Total number of locations
LEFT JOIN (
    SELECT [GEBIEDSCODE]
      ,count(*) AS NUMBER_OF_RWZI
    FROM [VWSSTATIC].[RWZI_INHIBITANTS_2021]
    WHERE GEBIEDSCODE like 'VR%'
    AND DATE_LAST_INSERTED = (select max(DATE_LAST_INSERTED) from [VWSSTATIC].[RWZI_INHIBITANTS_2021])
    AND PERCENTAGE != 0
    GROUP BY GEBIEDSCODE
) C
ON B.GEBIEDSCODE = C.GEBIEDSCODE
group by WEEK_UNIX, B.GEBIEDSCODE
END
GO

> MUNCIPALITIES

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER PROCEDURE [dbo].[SP_SEWER_GM]
AS
BEGIN
    -- Calculate percentages and inhabitants for each RWZI for each GM
    WITH RWZI_CTE AS (
            SELECT
            [RWZI_CODE]     AS CODE_RWZI,
            [STARTDATUM]    AS [STARTDATUM],
            [EINDDATUM]     AS [EINDDATUM],
            [INWONERS]      AS INHABITANTS,
            [REGIO_CODE]    AS GEBIEDSCODE,
            [AANDEEL]       AS [PERCENTAGE],
            [INWONERS] * ([AANDEEL]/100) AS [COVERED_INHABITANTS]
        FROM [VWSSTATIC].[CBS_POPULATION_RWZI]
        WHERE DATE_LAST_INSERTED = (select max(DATE_LAST_INSERTED) from [VWSSTATIC].[CBS_POPULATION_RWZI])
            AND REGIO_TYPE = 'GM'
    )

    -- Calculate average per RWZI per week
    ,BASE_CTE AS (
        SELECT T0.*
        ,T1.GEBIEDSCODE
        ,T1.[COVERED_INHABITANTS] AS APPLICABLE_INHABITANTS
        ,(T0.AVG_RNA_FLOW_PER_100000_PER_RWZI / 100000 ) * T1.[COVERED_INHABITANTS] AS AVG_RNA_FLOW_PER_RWZI
        FROM (
            SELECT
            SB.RWZI_AWZI_CODE,
            WEEK_UNIX, --Matches the monday (first day) of the week
            [dbo].[CONVERT_UNIX_TO_DATETIME](WEEK_UNIX)   AS WEEK_START,
            AVG(RNA_FLOW_PER_100000) / 1.0E+11 as AVG_RNA_FLOW_PER_100000_PER_RWZI
            , count(*) AS NUMBER_OF_MEASUREMENTS
            FROM VWSDEST.SEWER_BASE SB
            where DATE_LAST_INSERTED = (select max(DATE_LAST_INSERTED) from VWSDEST.SEWER_BASE)
            -- keep only the first measurement (some duplicate measurements per day per RWZI are present)
            and COUNT_DAILY_MEASUREMENT=1
            -- Take the average of the measurements for 1 RWZI for 1 week
            group by RWZI_AWZI_CODE, WEEK_UNIX
        ) AS T0
        -- Add percentages and inhabitants for each RWZI for each VR
        LEFT JOIN RWZI_CTE AS T1
            ON  T0.RWZI_AWZI_CODE = T1.CODE_RWZI
            AND T0.WEEK_START >= T1.STARTDATUM AND T0.WEEK_START < T1.EINDDATUM
            AND T1.PERCENTAGE > 0
    )

    -- Calculate the total amount of locations per GM per week.
    ,WEEK_STARTS_PER_RWZI AS (
        SELECT WEEK_START
        ,GEBIEDSCODE
        ,COUNT(CODE_RWZI) AS TOTAL_LOCATIONS
        FROM (
            SELECT DISTINCT WEEK_START
                FROM BASE_CTE
        ) A
        LEFT JOIN RWZI_CTE R
        ON A.WEEK_START >= R.STARTDATUM AND A.WEEK_START < R.EINDDATUM
        AND R.PERCENTAGE > 0
        GROUP BY WEEK_START, GEBIEDSCODE
    )

    INSERT INTO VWSDEST.SEWER_MEASUREMENTS_PER_MUNICIPALITY(
            WEEK_UNIX
        ,   GMCODE
        ,   AVERAGE_RNA_FLOW_PER_100000
        ,   NUMBER_OF_MEASUREMENTS
        ,   NUMBER_OF_LOCATIONS
        ,   TOTAL_LOCATIONS
    )

    SELECT
        WEEK_UNIX
        ,A.GEBIEDSCODE
        ,AVERAGE_RNA_FLOW_PER_100000
        ,NUMBER_OF_MEASUREMENTS
        ,NUMBER_OF_LOCATIONS
        ,B.TOTAL_LOCATIONS
    FROM (
        SELECT
            WEEK_UNIX
        ,   WEEK_START
        ,   GEBIEDSCODE
        ,   ROUND((SUM(AVG_RNA_FLOW_PER_RWZI) / SUM(APPLICABLE_INHABITANTS)) * 100000 , 2) as AVERAGE_RNA_FLOW_PER_100000
        ,   SUM(NUMBER_OF_MEASUREMENTS) as NUMBER_OF_MEASUREMENTS
        ,   count(*) AS NUMBER_OF_LOCATIONS
        FROM BASE_CTE
        group by WEEK_UNIX, WEEK_START, GEBIEDSCODE
    ) A
    LEFT JOIN WEEK_STARTS_PER_RWZI B
    ON A.WEEK_START = B.WEEK_START
    AND A.GEBIEDSCODE = B.GEBIEDSCODE
END
GO

> SEWER PROLONGATED

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER PROCEDURE [DBO].[SP_SEWER_PROLONGATED]
AS
BEGIN
WITH BASE_CTE AS (

    SELECT 
        WEEK
        ,DATE_MEASUREMENT
        ,RWZI_AWZI_CODE 
        ,RWZI_AWZI_NAME
        ,RNA_FLOW_PER_100000 
        ,RNA_FLOW_PER_100000 / 1.0E+11 AS RNA_FLOW_PER_100000_PER_RWZI
    FROM VWSDEST.SEWER_BASE
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.SEWER_BASE)
        -- KEEP ONLY THE FIRST MEASUREMENT (SOME DUPLICATE MEASUREMENTS PER DAY PER RWZI ARE PRESENT)
        AND COUNT_DAILY_MEASUREMENT=1
)
,
RWZI_CTE AS (
    -- ALLOCATION TABLE WITH DATA ON GM AND VR LEVEL
    SELECT 
        [RWZI_CODE],
        [RWZI_NAAM],
        [STARTDATUM],
        [EINDDATUM],
        [INWONERS],
        [REGIO_TYPE],
        [REGIO_CODE],
        [REGIO_NAAM],
        [AANDEEL],
        [INWONERS] * ([AANDEEL]/100) AS [COVERED_INHABITANTS]
    FROM VWSSTATIC.CBS_POPULATION_RWZI
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_RWZI)
        AND REGIO_TYPE = 'VR' -- GM WILL NOT BE USING THIS METHOD.
    
    UNION ALL

        SELECT DISTINCT
        [RWZI_CODE],
        [RWZI_NAAM],
        [STARTDATUM],
        [EINDDATUM],
        [INWONERS],
        'NL'            AS [REGIO_TYPE],
        'NL'            AS [REGIO_CODE],
        'Netherlands'   AS [REGIO_NAAM],
        100.00          AS [AANDEEL],
        [INWONERS]      AS [COVERED_INHABITANTS]
    FROM VWSSTATIC.CBS_POPULATION_RWZI
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_RWZI)
        AND REGIO_TYPE = 'VR' -- CHOOSE EITHER VR OR GM FOR RARE CASES WHERE INHABITANTS DO NOT ALIGN SUCH AS RWZI 9041

),
GRID_CTE AS (
    -- USE CALENDAR TABLE TO CREATE A RECORD FOR EACH POSSIBLE COMBINATION OF DATE, RWZI AND REGION
SELECT  
         T0.DATE_MEASUREMENT
        ,T1.RWZI_CODE
        ,T1.[REGIO_CODE]
        ,T1.[REGIO_NAAM]
    FROM 
        (
            SELECT CAST([DAY] AS DATETIME) AS  [DATE_MEASUREMENT] FROM [VWSSTATIC].[CALENDAR]
                WHERE   [DAY]>= (SELECT MIN(DATE_MEASUREMENT) FROM BASE_CTE)
                AND [DAY]<= (SELECT MAX(DATE_MEASUREMENT) FROM BASE_CTE)
        ) AS T0
    CROSS JOIN 
        (SELECT DISTINCT 
             [RWZI_CODE]
            ,[REGIO_CODE]
            ,[REGIO_NAAM]
        FROM RWZI_CTE) AS T1
    --ADD ADDITIONAL (DE-DUPLICATED) INFO TO RWZI AND REGIO DIMENSIONS
),
DAILY_CTE AS (
    SELECT 
         T0.WEEK
        ,T0.DATE_MEASUREMENT
        ,T0.RWZI_AWZI_CODE
        ,T0.RWZI_AWZI_NAME
        -- ,T0.RNA_FLOW_PER_100000 
        ,T0.RNA_FLOW_PER_100000_PER_RWZI
        ,T1.STARTDATUM
        ,T1.EINDDATUM
        ,T1.REGIO_CODE
        ,T1.AANDEEL
        ,T1.INWONERS
        ,T1.COVERED_INHABITANTS
    FROM BASE_CTE AS T0
    LEFT JOIN RWZI_CTE AS T1
        ON  T0.RWZI_AWZI_CODE = T1.RWZI_CODE
        AND T1.STARTDATUM <= T0.DATE_MEASUREMENT AND T1.EINDDATUM > T0.DATE_MEASUREMENT
),
SPARSE_CTE AS
(
    SELECT 
         T0.DATE_MEASUREMENT
        ,T0.RWZI_CODE
        ,T0.[REGIO_CODE]
        ,T0.[REGIO_NAAM]
        ,'|' AS [|]
        ,T1.WEEK
        ,T1.RWZI_AWZI_NAME
        ,T1.RNA_FLOW_PER_100000_PER_RWZI AS RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL
        ,T1.AANDEEL
        ,T1.INWONERS
        ,T1.COVERED_INHABITANTS
        --DETERMINE THE LAST DATE THAT CONTAINED VALUES FOR THIS RWZI AND REGION
        ,MAX(CASE WHEN T1.REGIO_CODE IS NOT NULL THEN T0.[DATE_MEASUREMENT] END)
            OVER (PARTITION BY T0.RWZI_CODE, T0.REGIO_CODE ORDER BY T0.[DATE_MEASUREMENT] ROWS 8 PRECEDING) AS PROLONGATE_DATE
    FROM GRID_CTE AS T0
    LEFT JOIN DAILY_CTE AS T1 ON T0.DATE_MEASUREMENT = T1.DATE_MEASUREMENT AND T0.RWZI_CODE = T1.RWZI_AWZI_CODE AND T0.REGIO_CODE = T1.REGIO_CODE
),
DENSE_CTE AS (
SELECT 
     *
     --ADD THE TOTAL NUMBER OF INHABITANTS IN THE REGION BASED ON THE PROLONGATED INHABITANT NUMBERS
    ,SUM(COVERED_INHABITANTS)                   OVER (PARTITION BY            REGIO_CODE, DATE_MEASUREMENT) AS REGION_COVERED_INHABITANTS
    FROM (
        --PROLONGATE THE VALUES: RETRIEVE THE ONLY VALUE DETERMINED FOR PROLONGATE DATE, RWZI AND REGION COMBINATION
        SELECT 
             DATE_MEASUREMENT
            ,RWZI_CODE
            ,[REGIO_CODE]
            ,[REGIO_NAAM]
            ,[|]
            ,MAX(RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL) OVER (PARTITION BY RWZI_CODE, REGIO_CODE, PROLONGATE_DATE) AS RNA_FLOW_PER_100000_PER_RWZI
            ,MAX(COVERED_INHABITANTS) OVER (PARTITION BY RWZI_CODE, REGIO_CODE, PROLONGATE_DATE) AS COVERED_INHABITANTS
            ,AANDEEL
            ,INWONERS
            -- ,COVERED_INHABITANTS
            ,PROLONGATE_DATE
        FROM SPARSE_CTE
        ) T0   
)
    INSERT INTO VWSDEST.SEWER_PROLONGATED(
         [DATE_MEASUREMENT]                     
        ,[REGIO_CODE]                           
        ,[WEEK]                                 
        ,[WEEK_UNIX]                            
        ,[AVERAGE_RNA_FLOW_PER_100000]          
        ,[REGION_COVERED_INHABITANTS]      
    )
    SELECT 
        [DATE_MEASUREMENT]
        ,[REGIO_CODE]
        ,DATEPART(ISO_WEEK, [DATE_MEASUREMENT]) AS WEEK
        ,[DBO].[CONVERT_DATETIME_TO_UNIX](DATEADD(WEEK, DATEDIFF(WEEK, 0, DATE_MEASUREMENT - 1), 0)) AS WEEK_UNIX
        ,CASE  
            WHEN [REGION_COVERED_INHABITANTS] != 0 THEN SUM( (COVERED_INHABITANTS * RNA_FLOW_PER_100000_PER_RWZI) ) / REGION_COVERED_INHABITANTS
            ELSE  SUM( ([COVERED_INHABITANTS] * RNA_FLOW_PER_100000_PER_RWZI) ) /1 
        END 
        AS [AVERAGE_RNA_FLOW_PER_100000]
        ,[REGION_COVERED_INHABITANTS]
    FROM DENSE_CTE
    GROUP BY DATE_MEASUREMENT, [REGIO_CODE], REGION_COVERED_INHABITANTS;
END;
GO

> SEWER PER RWZI GM VR

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER PROCEDURE [dbo].[SP_SEWER_MEASUREMENTS_PER_RWZI_GM_VR]
AS
BEGIN

WITH BASE_CTE AS (
SELECT 
       [WEEK]
      ,[DATE_MEASUREMENT]
      ,[RWZI_AWZI_CODE]
      ,[RWZI_AWZI_NAME]
      ,CAST(RNA_FLOW_PER_100000 / 1.0E+11 AS DECIMAL(16,2)) AS RNA_FLOW_PER_100000
  FROM [VWSDEST].[SEWER_BASE]
  WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[SEWER_BASE])
)
,
RWZI_POPULATION_CTE AS (
    SELECT
        REGIO_CODE 
        ,RWZI_CODE
        ,REGIO_TYPE
        ,STARTDATUM
        ,EINDDATUM
    FROM VWSSTATIC.CBS_POPULATION_RWZI WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_RWZI)
)
INSERT INTO VWSDEST.SEWER_MEASUREMENTS_PER_RWZI_GM_VR
(
        [DATE_MEASUREMENT]		,	
	    [RWZI_AWZI_CODE]        ,
	    [RWZI_AWZI_NAME]        ,
	    RNA_FLOW_PER_100000     ,					
        REGIO_TYPE			    ,									
        REGIO_CODE			    ,										
        WEEK				    ,														
        DATE_UNIX               ,        
        DATE_START_UNIX         ,        
        DATE_END_UNIX                 
)
SELECT        
         T0.[DATE_MEASUREMENT]
        ,T0.[RWZI_AWZI_CODE]
        ,T0.[RWZI_AWZI_NAME]
        ,T0.[RNA_FLOW_PER_100000]
        ,T1.REGIO_TYPE
        ,T1.REGIO_CODE
        ---Dates for view
        ,WEEK
        ,dbo.CONVERT_DATETIME_TO_UNIX(T0.[DATE_MEASUREMENT])                    AS DATE_UNIX
        ,dbo.CONVERT_DATETIME_TO_UNIX(dbo.WEEK_START(T0.[DATE_MEASUREMENT]))    AS DATE_START_UNIX
        ,dbo.CONVERT_DATETIME_TO_UNIX(dbo.WEEK_END(T0.[DATE_MEASUREMENT]))      AS DATE_END_UNIX
    FROM BASE_CTE AS T0
        LEFT JOIN RWZI_POPULATION_CTE T1 
            ON T0.RWZI_AWZI_CODE = T1.RWZI_CODE
            AND T1.STARTDATUM <= T0.DATE_MEASUREMENT AND T1.EINDDATUM > T0.DATE_MEASUREMENT



END;
GO


> SEWER PER RWZI MINIMAL

In [ ]:
-- 1) CREATE VIEW(S)..... 
CREATE OR ALTER PROCEDURE [dbo].[SP_SEWER_MEASUREMENTS_PER_RWZI_MINIMAL]
AS
BEGIN

    INSERT INTO
        [VWSDEST].[SEWER_MEASUREMENTS_PER_RWZI_MINIMAL] (
            DATE_UNIX, 
            DATE_START_UNIX, 
            DATE_END_UNIX,WEEK,
            RWZI_AWZI_CODE,
            RWZI_AWZI_NAME,
            GMCODE,
            RNA_NORMALIZED
        )
    SELECT
        [DATE_MEASUREMENT_UNIX] AS DATE_UNIX
        ,dbo.CONVERT_DATETIME_TO_UNIX(dbo.WEEK_START(dbo.CONVERT_UNIX_TO_DATETIME(DATE_MEASUREMENT_UNIX))) AS DATE_START_UNIX
        ,dbo.CONVERT_DATETIME_TO_UNIX(dbo.WEEK_END(dbo.CONVERT_UNIX_TO_DATETIME(DATE_MEASUREMENT_UNIX))) AS DATE_END_UNIX
        ,dbo.CONVERT_UNIX_TO_ISO_WEEKNUMBER(DATE_MEASUREMENT_UNIX) as WEEK
        ,[RWZI_AWZI_CODE]
        ,[RWZI_AWZI_NAME]
        ,GEBIEDSCODE AS [GMCODE]
        ,RNA_FLOW_PER_100000 AS RNA_NORMALIZED
    FROM VWSDEST.SEWER_MEASUREMENTS_PER_RWZI T1
    LEFT JOIN VWSSTATIC.RWZI_INHIBITANTS_2021 T2
    ON T1.RWZI_AWZI_CODE = T2.CODE_RWZI
    AND T2.PERCENTAGE != 0
    AND GEBIEDSCODE like 'GM%'
    AND T2.DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.RWZI_INHIBITANTS_2021 )
    WHERE T1.DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.SEWER_MEASUREMENTS_PER_RWZI)
    AND RWZI_AWZI_CODE != '0'
    AND DATE_MEASUREMENT > '2020-01-01'
END;
GO